## 线性规划的实现

### 数据预处理

In [1]:
%matplotlib inline
import torch
import numpy
import random
def generate_data(w,b,num_examples):
    x=torch.normal(0,1,(num_examples,len(w)))
    y=torch.matmul(x,w)+b
    y+=torch.normal(0,0.01,y.shape)
    return x,y.reshape((-1,1))
true_w=torch.tensor([2,-3.4])
true_b=4.2
features,labels=generate_data(true_w,true_b,1000)

### 生成训练集和验证集

In [2]:
def data_iter(feature,label,batch_size):
    num_examples=len(feature)#对于tensor来说，len测量行数
    data_no=list(range(num_examples))
    random.shuffle(data_no)
    for i in range(0,num_examples,batch_size):
        batch_no=list(data_no[i:min(i+batch_size,num_examples)])
        yield feature[batch_no],label[batch_no]

### 定义模型参数（w、b）、模型、损失函数、优化算法

In [3]:
w=torch.normal(0,1,size=(2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

def LinearRegression(X,w,b):
    return torch.matmul(X,w)+b

def squared_loss(y_hat,y):
    return (y_hat-y.reshape(y_hat.shape))**2/2

def sgd(params,learning_rate,batch_size):
    with torch.no_grad():
        for param in params:
            param-=learning_rate*param.grad/batch_size
            param.grad.zero_()

### 训练

In [4]:
batch_size=10
lr=0.03
num_epochs=3
for epoch in range(num_epochs):
    for X,y in data_iter(features,labels,batch_size):
        y_hat=LinearRegression(X,w,b)
        loss = squared_loss(y_hat,y)
        loss.sum().backward()
        sgd([w,b],lr,batch_size)
    with torch.no_grad():
        loss = squared_loss(LinearRegression(features,w,b),labels)
        print(f'epoch{epoch} loss: {loss.mean()}')
    print(w,b)


epoch0 loss: 0.03794435039162636
tensor([[ 1.8432],
        [-3.2762]], requires_grad=True) tensor([4.0066], requires_grad=True)
epoch1 loss: 0.00013772603415418416
tensor([[ 1.9917],
        [-3.3938]], requires_grad=True) tensor([4.1910], requires_grad=True)
epoch2 loss: 4.793088010046631e-05
tensor([[ 1.9994],
        [-3.4001]], requires_grad=True) tensor([4.1997], requires_grad=True)
